GitHub<br>
https://github.com/microsoft/GLIP<br>
論文<br>
https://arxiv.org/abs/2206.05836<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/GLIP_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境セットアップ

## Githubからソースコード取得

In [ ]:
%cd /content

!git clone https://github.com/microsoft/GLIP.git

## ライブラリのインストール

In [ ]:
%cd /content/GLIP

!pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install einops shapely timm yacs tensorboardX ftfy prettytable pymongo
!pip install transformers

!python setup.py build develop --user

## ライブラリのインポート

In [ ]:
%cd /content/GLIP

import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

import requests
from io import BytesIO
from PIL import Image
import numpy as np
pylab.rcParams['figure.figsize'] = 20, 12
from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.engine.predictor_glip import GLIPDemo

# 学習済みモデルのセットアップ

In [ ]:
%cd /content/GLIP
!mkdir MODEL

!wget -c https://penzhanwu2bbs.blob.core.windows.net/data/GLIPv1_Open/models/glip_tiny_model_o365_goldg_cc_sbu.pth \
      -O MODEL/glip_tiny_model_o365_goldg_cc_sbu.pth


# Utility関数定義

In [ ]:
def load(url):
    """
    Given an url of an image, downloads the image and
    returns a PIL image
    """
    response = requests.get(url)
    pil_image = Image.open(BytesIO(response.content)).convert("RGB")
    # convert to BGR format
    image = np.array(pil_image)[:, :, [2, 1, 0]]
    return image

def imshow(img, caption):
    plt.imshow(img[:, :, [2, 1, 0]])
    plt.axis("off")
    plt.figtext(0.5, 0.09, caption, wrap=True, horizontalalignment='center', fontsize=20)

# build model

In [ ]:
%cd /content/GLIP

config_file = "configs/pretrain/glip_Swin_T_O365_GoldG.yaml"
weight_file = "MODEL/glip_tiny_model_o365_goldg_cc_sbu.pth"

cfg.local_rank = 0
cfg.num_gpus = 1
cfg.merge_from_file(config_file)
cfg.merge_from_list(["MODEL.WEIGHT", weight_file])
cfg.merge_from_list(["MODEL.DEVICE", "cuda"])

glip_demo = GLIPDemo(
  cfg,
  min_image_size=800,
  confidence_threshold=0.7,
  show_mask_heatmaps=False,
)

# image, captionの設定

In [ ]:
img_url = 'https://www.pakutaso.com/shared/img/thumb/smIMGL4174_TP_V.jpg' #@param {type:'string'}
caption = 'yellow fruit' #@param {type:'string'}

# Object detection

In [ ]:
glip_demo.color = 128

In [ ]:
image = load(img_url)
result, _ = glip_demo.run_on_web_image(image, caption, 0.5)
imshow(result, caption)